## Notebook for use case digital posting assistant - stage1
### Module B accounting assignment
#### Ojectives
- In this module we will develop the accounting assignment regarding the given booking information from user

#### Processing steps from concept
B0 - preparation

B1 - input: get booking information business case from user

B2 - retrieve: retrieve relevant accounting assignments from SAP HANA vector database 

B3 - answer: create relevant accounting assigments for business case with LLM

B4 - output: give answer with accounting assigments to user

### B0 - Setup and configuration Modul B

The following setup-steps where processed:

* B0.0 Start SAP instances
* B0.1 install py-packages
* B0.2 load env-variables from config.json-file
* B0.3 setup and test connection to HANA DB
* B0.4 setup LLM-Connection to SAP AI-HUB
* B0.5 setup embedding-model
* B0.6 create SAP HANA-VectorStore interface with table
* B0.7 setup class user interface

### B0.0 Start SAP Instances

- Start BTP Cockpit
- Start SAP Build Dev Space
- Start HANA DB

In [ ]:
# B0.1 install py-packages
# RESET KERNEL AFTER INSTALLATION

%pip install --quiet --upgrade pip

%pip install --quiet hdbcli --break-system-packages
%pip install --quiet generative-ai-hub-sdk[all] --break-system-packages
%pip install --quiet folium --break-system-packages
%pip install --quiet ipywidgets --break-system-packages
%pip install --quiet pypdf
%pip install --quiet -U ipykernel
%pip install --quiet hana-ml
%pip install --quiet langchain
%pip install --quiet hdbcli
%pip install --quiet sqlalchemy-hana
%pip install --quiet transformers # for anthropic models
print("py-packages installed!")

In [55]:
# B0.2 load env-variables from config.json-file
# This script loads environment variables from a JSON configuration file
# and sets them in the current environment. It raises an error if the file does not exist
# or if the JSON file is malformed.

import json
import os

def load_env_variables(config_file):
    """
    Load environment variables from a JSON configuration file.

    Args:
        config_file (str): Path to the JSON configuration file.

    Returns:
        dict: A dictionary containing the environment variables.
    """
    if not os.path.exists(config_file):
        raise FileNotFoundError(f"The configuration file {config_file} does not exist.")
    
    try:
        with open(config_file, 'r') as file:
            env_variables = json.load(file)
    except json.JSONDecodeError as e:
        raise ValueError(f"Error decoding JSON from the configuration file {config_file}: {e}")
    
    for key, value in env_variables.items():
        # Convert non-string values to strings before setting them in os.environ
        if isinstance(value, dict):
            value = json.dumps(value)  # Convert dictionaries to JSON strings
        os.environ[key] = str(value)
    
    return env_variables

# Load environment variables from the specified JSON configuration file
config_file = "/home/user/.aicore/config.json"
try:
    env_variables = load_env_variables(config_file)
    print(f"Loaded environment variables: {env_variables}")
except (FileNotFoundError, ValueError) as e:
    print(e)

Loaded environment variables: {'AICORE_BASE_URL': 'https://api.ai.prod-eu20.westeurope.azure.ml.hana.ondemand.com', 'AICORE_AUTH_URL': 'https://sap-genai.authentication.eu20.hana.ondemand.com', 'AICORE_CLIENT_ID': 'sb-7a06c038-4de3-4eb6-9fc5-552da47413e6!b78145|xsuaa_std!b57362', 'AICORE_CLIENT_SECRET': 'a657d1f4-3c99-42d7-9b74-103b3d8b701e$q9XVOJA5DFfy6UxcfhbUB1vk8QB0g6anRGT5SQuqOTw=', 'AICORE_RESOURCE_GROUP': 'default', 'AICORE_DEPLOYMENT_MODEL': 'gpt-4o', 'AICORE_DEPLOYMENT_MODEL_EMBEDDING': 'text-embedding-3-large', 'hdb_host_address': 'bfff8255-c34a-41cb-a822-bf9b5f56fb16.hna0.prod-eu20.hanacloud.ondemand.com', 'hdb_user': 'DBADMIN', 'hdb_password': 'DB@hmx04', 'hdb_port': '443', 'hdb_table_name': 'ACCOUNTING_ASSIGN_SUPPORT_TABLE_DBADMIN'}


In [ ]:
# B0.2 Test connection with env-Variables to SAP AI core

from gen_ai_hub.proxy.native.openai import embeddings
import os

# Ensure the correct model name is used
model_embedding_name = os.getenv("AICORE_DEPLOYMENT_MODEL_EMBEDDING")  # Default to a valid model
try:
    response = embeddings.create(
        input="SAP Generative AI Hub is awesome!",
        # deployment_id=deployment_id, # Uncomment if using a  model deployment ID
        # model_id=model_id,  # Uncomment if using a  model ID
        model_name=model_embedding_name,  # Uncomment if using a  model name
        # model_version="latest",  # Uncomment if using a specific version
        # model_type="text",  # Uncomment if using a specific model type
        #nmodel_version="latest",  # Uncomment if using the latest version
        #nmodel_type="text-embedding",  # Uncomment if using a specific model type
    )
    print(response.data)
except ValueError as e:
    print(f"Error: {e}")
    print("Ensure the model name matches an existing deployment in SAP AI Hub.")

In [ ]:
# B0.3 Setup and test connection to HANA DB

import os
# from hana_ml import ConnectionContext
from hdbcli import dbapi

# Fetch environment variables
hdb_host_address = os.getenv("hdb_host_address")
hdb_user = os.getenv("hdb_user")
hdb_password = os.getenv("hdb_password")
hdb_port = os.getenv("hdb_port")

# Debugging: Print non-sensitive environment variables
print(f"hdb_host_address: {hdb_host_address}")
print(f"hdb_user: {hdb_user}")
print(f"hdb_port: {hdb_port}")

# Ensure variables are defined
if not all([hdb_host_address, hdb_user, hdb_password, hdb_port]):
    raise ValueError("One or more HANA DB connection parameters are missing.")

# Convert port to integer
hdb_port = int(hdb_port)

hana_connection = dbapi.connect(
    address=hdb_host_address,
    port=hdb_port,
    user=hdb_user,
    password=hdb_password,
    #encrypt=True
    autocommit=True,
    sslValidateCertificate=False,
)

In [56]:
#B0.4 Setup LLM-Connection to SAP AI-HUB (Model)

import os
import re
import re
import dotenv
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from gen_ai_hub.proxy.langchain.openai import OpenAI
from gen_ai_hub.proxy.langchain.amazon import ChatBedrock
from gen_ai_hub.proxy.langchain.amazon import ChatBedrockConverse
from gen_ai_hub.proxy.langchain.init_models import init_llm

# Supported models 
# supportedModels openAI: 
# gpt-35-turbo-16k, o3-mini, gpt-35-turbo, 
# gpt-4-32k, gpt-4, gpt-4o, gpt-4o-mini, 
# text-embedding-ada-002, text-embedding-3-large, text-embedding-3-small
# supportedModels Amazon Bedrock:
# supportedModels Amazon Bedrock: 
# anthropic--claude-3-haiku, anthropic--claude-3-opus, anthropic--claude-3-sonnet, anthropic--claude-3.5-sonnet, 
# amazon--titan-text-express, amazon--titan-text-lite, amazon--titan-embed-text
# supportedModels GCP Vertex AI:
# text-bison, chat-bison, 
# textembedding-gecko-multilingual, textembedding-gecko, 
# gemini-1.0-pro, gemini-1.5-pro, gemini-1.5-flash

# Lade aicore_model_name aus der Umgebungskonfiguration
aicore_model_name = str(os.getenv("AICORE_DEPLOYMENT_MODEL"))


# Überprüfe, ob die Variable definiert ist
if not aicore_model_name:
    raise ValueError(f"""Parameter LLM-Model-Name {aicore_model_name} fehlt in der Umgebungskonfiguration.""")

# Set LLM with model from config.json
llm = None
llm = init_llm(
    model_name = aicore_model_name, 
    temperature = 0
    )

# check if the LLM is set
if not llm:
    raise ValueError(f"""Parameter LLM-Model-Name {aicore_model_name} fehlt in der Umgebungskonfiguration.""")
else:
    print(f"""Parameter LLM-Model-Name: {aicore_model_name} wurde erfolgreich geladen.""")


Parameter LLM-Model-Name: gpt-4o wurde erfolgreich geladen.


In [57]:
#B0.4 Check Setup LLM-Connection to SAP AI-HUB (Model)

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from gen_ai_hub.proxy.langchain.init_models import init_llm

template = """Question: {question}
    Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=['question'])
question = 'What is a ai-agent?'

# llm = init_llm('anthropic--claude-3-opus', max_tokens=300)
chain = prompt | llm | StrOutputParser()
response = chain.invoke({'question': question})
print(response)

An AI agent is a software entity that performs tasks autonomously using artificial intelligence techniques. To understand what an AI agent is, let's break it down step by step:

1. **Definition of an Agent**: In general terms, an agent is something that acts on behalf of another. In computing, an agent is a program that can perform tasks autonomously, often in a specific environment.

2. **Artificial Intelligence**: AI refers to the simulation of human intelligence processes by machines, especially computer systems. These processes include learning, reasoning, problem-solving, perception, and language understanding.

3. **AI Agent Characteristics**:
   - **Autonomy**: AI agents operate without direct human intervention, making decisions based on their programming and the data they receive.
   - **Reactivity**: They perceive their environment and respond to changes in a timely manner.
   - **Proactivity**: AI agents can take initiative and perform goal-directed behavior.
   - **Social A

In [58]:
# B0.5 setup embedding-model
# Use the embedding models from SAP AI-hub for embedding.

# Initialize embeddings

from gen_ai_hub.proxy.langchain.init_models import init_embedding_model

ai_core_embedding_model_name = str(os.getenv('AICORE_DEPLOYMENT_MODEL_EMBEDDING'))
 
try:
    embeddings = init_embedding_model(ai_core_embedding_model_name)
    print("Embedding model initialized successfully: ", ai_core_embedding_model_name)
except Exception as e:
    print("Embedding model not initialized.")
    print(e)

Embedding model initialized successfully:  text-embedding-3-large


In [ ]:
# B0.6 - create SAP HANA-VectorStore interface with table 
# check table content with sap-hana-database explorer: select * from ACCOUNTING_ASSIGN_SUPPORT_TABLE_DBADMIN

from langchain_community.vectorstores.hanavector import HanaDB

vector_table_name = str(os.getenv('hdb_table_name')) 

hana_database = HanaDB(
    embedding = embeddings, 
    connection = hana_connection, 
    table_name = vector_table_name
)

try:
    print(f"""
    Successfully created SAP HANA VectorStore interface: {hana_database.connection}
    and SAP HANA table: {vector_table_name}.
    """)
except Exception as e:
    print(e)


In [ ]:
# check B0.6 - query to the SAP HANA table to verify embeddings

cursor = hana_connection.cursor()
sql = f'SELECT VEC_TEXT, TO_NVARCHAR(VEC_VECTOR) FROM "{hana_database.table_name}"'

cursor.execute(sql)
vectors = cursor.fetchall()

print(vectors[5:10])

# for vector in vectors:
#     print(vector)

In [ ]:
# B0.7 - setup class user interface
# user interface version 2 (class) - function B1.1 : get booking information business case from user
# UI-class for storing input booking information business case from user 


import ipywidgets as widgets
from IPython.display import display, HTML

class InputManager:
    def __init__(self):
        self.input_text = ""
        self.history = []
        
        # Button-Object
        self.button = widgets.Button(
            description='Senden',
            button_style='primary',
            layout=widgets.Layout(
                width='150px',
                height='40px',
                margin='0 0 10px 0'
            )
        )
        
        # Input-Field with Header
        input_label = widgets.HTML(
            value="<h3 style='margin-bottom: 10px; color: #333;'>Eingabe Buchungsinformationen</h3>"
        )
        
        # Widget for Text-input
        self.text_input = widgets.Textarea(
            value='',
            placeholder='Geben Sie hier Ihre Buchungsinformationen ein...',
            disabled=False,
            layout=widgets.Layout(
                width='800px',
                height='200px',
                # overflow='hidden'  # Verhindert Scrolling
            )
        )
        
        # Output-Field with Header
        output_label = widgets.HTML(
            value="<h3 style='margin: 20px 0 10px 0; color: #333;'>Ausgabe Kontierungsinformationen</h3>"
        )
        
        # Output-Field function
        self.output = widgets.HTML(
            value="""
            <div style='
                border: 1px solid #ddd;
                padding: 10px;
                background-color: white;
                height: 300px;
                width: 800px;
            '>
                <p>Kontierungsinformationen werden hier ausgegeben ...</p>
            </div>
            """,
            layout=widgets.Layout(
                width='800px',
                height='150px',
                # overflow='hidden'  # Verhindert Scrolling
            )
        )
        
        # Button-Callback
        self.button.on_click(self.on_button_click)
        
        # vertical Layout UI (VBox)
        content = widgets.VBox([
            input_label,
            self.button,
            self.text_input,
            output_label,
            self.output
        ], layout=widgets.Layout(
            width='900px',
            align_items='flex-start'
        ))
        
        # Container UI (VBox)
        self.container = widgets.VBox([
            widgets.HTML("<h1>Digitaler Buchungsassistent - Kontierungshilfe</h1>"),
            content
        ], layout=widgets.Layout(
            width='900px',
            padding='20px'
        ))
    
    def on_button_click(self, b):
        """Wird aufgerufen, wenn der Button geklickt wird"""
        self.input_text = self.text_input.value
        self.history.append(self.input_text)
        
        # Output-field refresh after click
        self.update_output(f"Eingabe: {self.input_text}")
        
        # clear input field after click
        # self.text_input.value = ''
    
    def update_output(self, text):
        """Ausgabefeld aktualisieren"""
        self.output.value = f"""
        <div style='
            border: 1px solid #ddd;
            padding: 10px;
            background-color: white;
            height: 300px;
            width: 800px;
            # overflow: hidden;
        '>
            <p>{text}</p>
        </div>
        """
    
    def display_widget(self):
        """Widget anzeigen"""
        display(self.container)
    
    def get_current_input(self):
        """Aktuelle Eingabe zurückgeben"""
        return self.input_text
    
    def get_history(self):
        """Eingabehistorie zurückgeben"""
        return self.history
    
    def save_history(self, filename='input_history.json'):
        """Historie in Datei speichern"""
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(self.history, f, ensure_ascii=False, indent=2)
    
    def load_history(self, filename='input_history.json'):
        """Historie aus Datei laden"""
        try:
            with open(filename, 'r', encoding='utf-8') as f:
                self.history = json.load(f)
        except FileNotFoundError:
            print("Keine gespeicherte Historie gefunden.")

# Instance creation
input_manager = InputManager()

# UI call
# input_manager.display_widget()

print("Instance UI created!")

### processing functions Modul B

* B1 - input: get booking information business case from user

* B2 - retrieve: retrieve relevant accounting assignments from SAP HANA vector database 

* B3 - answer: create relevant accounting assigments for business case with LLM

* B4 - output: give answer with accounting assigments to user

### function B1: get booking information business case from user


### function B1: get booking information business case from user
### user interface

input-example Booking information business case from user: 

- Es soll eine Rückstellung für folgenden Geschäftsfall gebucht werden.
- Es konnte eine Instandhaltung einer Maschine bis zum Jahresende des laufenden Geschäftsjahres nicht durchgeführt werden. Aus diesem Grund sind Rückstellungen im laufenden Jahr zu bilden. Aufgrund der Leistungserbringung im kommenden Geschäftsjahr ist diese Rückstellung im kommenden Geschäftsjahr aufzulösen.
- Der Kostenvoranschlag des Anbieters für die Instandhaltungsleistung beträgt im laufenden GEschäftsjahr 20.000 Euro netto inkl. evtl. Nachlässe und Nebenkosten. 
- Die Instandhaltung wird im folgenden Geschäftsjahr durchgeführt und abgerechnet. Die Rechnung enthält folgende Informationen: Lieferant A, Rechnungsbetrag Netto 19.500 EUR, Vorsteuer-Betrag von 3.705 EUR.
Für diese Abrechnung muss die Rückstellung im kommenden Geschäftsjahr aufgelöst werden.


In [ ]:
# function B1: get booking information business case from user
# function B1.1: get booking information business case from user
# the function show ui and store input from user

# Input-variables: none
# Output-variables: ui_input

input_manager.display_widget()
ui_input = input_manager.get_current_input()
print(f"Input from UI: {ui_input}")


### function B2: retrieve relevant accounting assignments from SAP HANA vector database
### function B3: create relevant accounting assigments for business case with LLM
#### Version 1 - RetrievalQA-Chain with MAP-REDUCE

In [73]:
# function B2 - retrieve: retrieve relevant accounting assignments from SAP HANA vector database 
# function B2.1.1- define MAP-Prompt (version 1 - origin from concept)

from langchain.prompts import PromptTemplate

map_prompt_template = """
# Kontierungsanalyse für Geschäftsfälle

Extrahiere kontierungsrelevante Informationen aus folgendem Geschäftsfall:

{context}

Gehe dabei Schritt für Schritt vor und achte auf die folgenden Punkte:
- Identifiziere die Kategorie des Geschäftsfalls    
- Bestimme die wesentlichen finanziellen Merkmale und Beträge
- Wende die Kontierungsregel an
- Ermittle konkrete Konto-Nummern und -Bezeichnungen aus dem Kontext
- Bestimme die exakten Buchungsbeträge (inkl. Steuern falls zutreffend)
- Identifiziere eventuelle Besonderheiten oder Ausnahmen
- Nutze folgenden Schritte des Analyseprozesses

## Analyseprozess:

### 1. Identifiziere die Geschäftsfall-Kategorie
- Identifiziere die Kategorie des Geschäftsfalls 
Geschäftsfall-Kategorie sind sind zum Beispiel:

|Geschäftsfall-Kategorie|
|-----------------------|
|Warenverkauf auf Rechnung|
|Wareneinkauf auf Rechnung|
|Zahlung an Lieferanten|
|Zahlungseingang von Kunden|
|Lohn- und Gehaltszahlungen|
|Anschaffung von Anlagevermögen|
|Abschreibungen|
|Bildung von Rückstellungen|
|Auflösung von Rückstellungen|
|Zahlung der Umsatzsteuer|

- Bestimme die wesentlichen finanziellen Merkmale und Beträge

### 2. Wende die Kontierungsregel an

Beachte die Grundprinzipien:
- Jede Buchung erfordert mindestens ein Soll- und Haben-Konto
- Summe Soll = Summe Haben

Ordne den Geschäftsfall anhand des Kontext einer dieser Geschäftsfall-Kategorien zu:

|Geschäftsfall-Kategorie|Konto Soll|Konto Haben|
|-----------------------|----------|-----------|
|Warenverkauf auf Rechnung|Forderungen aus Lieferungen und Leistungen|Umsatzerlöse und Umsatzsteuer|
|Wareneinkauf auf Rechnung|Wareneinsatz/Materialaufwand und Vorsteuer|Verbindlichkeiten aus Lieferungen und Leistungen|
|Zahlung an Lieferanten|Verbindlichkeiten aus Lieferungen und Leistungen|Bank oder Kasse|
|Zahlungseingang von Kunden|Bank oder Kasse|Forderungen aus Lieferungen und Leistungen|
|Lohn- und Gehaltszahlungen|Personalaufwand|Bank und diverse Verbindlichkeiten (Lohnsteuer, Sozialversicherung)|
|Anschaffung von Anlagevermögen|Anlagevermögen und Vorsteuer|Bank oder Verbindlichkeiten|
|Abschreibungen|Abschreibungsaufwand|Kumulierte Abschreibungen (Wertberichtigung Anlagevermögen)|
|Bildung von Rückstellungen|Aufwand für Rückstellungen|Rückstellungen|
|Auflösung von Rückstellungen|Rückstellungen|Sonstige betriebliche Erträge oder Aufwandskonto|
|Zahlung der Umsatzsteuer|Umsatzsteuer|Bank|


### 3. Extrahiere die relevante Kontierung aus dem Kontext
- Ermittle konkrete Konto-Nummern und -Bezeichnungen aus dem Kontext
- Bestimme die exakten Buchungsbeträge (inkl. Steuern falls zutreffend)
- Identifiziere eventuelle Besonderheiten oder Ausnahmen

### 4. Prüfe die Qualität des Ergebnisses
- Prüfe die doppelte Buchführung (Betrag Soll = Betrag Haben)
- Stelle Übereinstimmung mit gesetzlichen Anforderungen sicher
- Verifiziere die inhaltliche Korrektheit der Kontierung


Frage: {question}
"""
print("MAP_Prompt set")

MAP_Prompt set


In [59]:
# function B2 - retrieve: retrieve relevant accounting assignments from SAP HANA vector database 
# function B2.1.2 - define MAP-Prompt (version 2 - finetuning)

from langchain.prompts import PromptTemplate

map_prompt_template = """
# Kontierungsanalyse für Geschäftsfälle

Extrahiere kontierungsrelevante Informationen aus folgendem Geschäftsfall:

{context}

## Analyseprozess:

### 1. Analyse der Geschäftsfall-Kategorie
- Identifiziere die Kategorie des Geschäftsfalls 
Geschäftsfall-Kategorie sind sind zum Beispiel:

|Geschäftsfall-Kategorie|
|-----------------------|
|Warenverkauf auf Rechnung|
|Wareneinkauf auf Rechnung|
|Zahlung an Lieferanten|
|Zahlungseingang von Kunden|
|Lohn- und Gehaltszahlungen|
|Anschaffung von Anlagevermögen|
|Abschreibungen|
|Bildung von Rückstellungen|
|Auflösung von Rückstellungen|
|Zahlung der Umsatzsteuer|

- Bestimme die wesentlichen finanziellen Merkmale und Beträge

### 2. Kontierungsregel anwenden
Beachte die Grundprinzipien:
- Jede Buchung erfordert mindestens ein Soll- und Haben-Konto
- Summe Soll = Summe Haben

Ordne den Geschäftsfall einer dieser Kategorien zu:

|Geschäftsfall-Kategorie|Konto Soll|Konto Haben|
|-----------------------|----------|-----------|
|Warenverkauf auf Rechnung|Forderungen aus Lieferungen und Leistungen|Umsatzerlöse und Umsatzsteuer|
|Wareneinkauf auf Rechnung|Wareneinsatz/Materialaufwand und Vorsteuer|Verbindlichkeiten aus Lieferungen und Leistungen|
|Zahlung an Lieferanten|Verbindlichkeiten aus Lieferungen und Leistungen|Bank oder Kasse|
|Zahlungseingang von Kunden|Bank oder Kasse|Forderungen aus Lieferungen und Leistungen|
|Lohn- und Gehaltszahlungen|Personalaufwand|Bank und diverse Verbindlichkeiten (Lohnsteuer, Sozialversicherung)|
|Anschaffung von Anlagevermögen|Anlagevermögen und Vorsteuer|Bank oder Verbindlichkeiten|
|Abschreibungen|Abschreibungsaufwand|Kumulierte Abschreibungen (Wertberichtigung Anlagevermögen)|
|Bildung von Rückstellungen|Aufwand für Rückstellungen|Rückstellungen|
|Auflösung von Rückstellungen|Rückstellungen|Sonstige betriebliche Erträge oder Aufwandskonto|
|Zahlung der Umsatzsteuer|Umsatzsteuer|Bank|


### 3. Relevante Kontierung extrahieren
- Ermittle konkrete Konto-Nummern und -Bezeichnungen aus dem Kontext
- Bestimme die exakten Buchungsbeträge (inkl. Steuern falls zutreffend)
- Identifiziere eventuelle Besonderheiten oder Ausnahmen

### 4. Qualitätsprüfung
- Prüfe die doppelte Buchführung (Betrag Soll = Betrag Haben)
- Stelle Übereinstimmung mit gesetzlichen Anforderungen sicher
- Verifiziere die inhaltliche Korrektheit der Kontierung

## Ausgabe:
- Geschäftsfall-Kategorie
- Exakte Kontierungsinformation mit Kontonummern und -bezeichnungen
- Soll-Haben-Beziehung mit Beträgen

Falls keine eindeutigen Kontierungsinformationen extrahierbar: "Keine ausreichenden Kontierungsinformationen im Kontext verfügbar."

Frage: {question}
"""
print("MAP_Prompt set")

MAP_Prompt set


In [63]:
# function B2 - retrieve: retrieve relevant accounting assignments from SAP HANA vector database 
# function B2.1.3 - define MAP-Prompt (version 3 - minimum)

# Beispiel für verbesserten MAP-Prompt mit Relevanzprüfung
map_prompt_template = """
Extrahiere kontierungsrelevante Informationen aus folgendem Kontext:

{context}

Wichtig: Prüfe zuerst die Relevanz des Kontexts:
1. Enthält der Kontext konkrete Informationen zum Geschäftsfall?
2. Sind Kontierungshinweise (Konten, Beträge, Buchungsregeln) enthalten?

Falls KEINE relevanten Informationen vorhanden sind, antworte nur mit:
"KEINE RELEVANTEN KONTIERUNGSINFORMATIONEN GEFUNDEN"

Falls relevante Informationen vorhanden sind:
1. Identifiziere die Geschäftsfall-Kategorie
2. Extrahiere die konkreten Konten (Soll/Haben)
3. Notiere die genauen Beträge

Gib deine Antwort in diesem Format zurück:
Geschäftsfall-Kategorie: [KATEGORIE]
Relevante Konten: [KONTEN]
Beträge: [BETRÄGE]
Buchungsregel: [REGEL]

Frage: {question}
"""

In [74]:
# function B2 - retrieve: retrieve relevant accounting assignments from SAP HANA vector database 
# function B2.1.4 - define REDUCE-Prompt

from langchain.prompts import PromptTemplate

combine_prompt_template = """
# Kontierungsergebnis-Aggregation

{summaries}

## Aufgabe:
Konsolidiere die identifizierten Kontierungsinformationen zum folgenden Geschäftsfall:

Frage: {question}

Gehe entsprechend der folgenden Schritte vor:

### Schritt 1 - Prüfe die Relevanz der Informationen
## Vorgehen:
1. Enthält der Kontext konkrete Informationen zum Geschäftsfall?
2. Sind Kontierungshinweise (Konten, Beträge, Buchungsregeln) enthalten?
Falls KEINE relevanten Informationen vorhanden sind, antworte nur mit: "Ich habe keine Kontierungsinformationen zum Geschäftsfall gefunden."
3. Entferne Duplikate und redundante Informationen
4. Priorisiere die relevantesten und spezifischsten Kontierungen
5. Strukturiere das Ergebnis klar und übersichtlich

### Schritt 2 - Gib die Ergebnisse aus

## Vorgehen:
1. Entferne Duplikate und redundante Informationen
2. Priorisiere die relevantesten und spezifischsten Kontierungen
3. Strukturiere das Ergebnis klar und übersichtlich

- Gib folgende Informationen des Ergebnisses aus:
    * Geschäftsfallbezeichnung
    * Geschäftsfall-Kategorie
    * Exakte Kontierungsinformation mit Kontonummern und -bezeichnungen
    * Soll-Haben-Beziehung mit Beträgen

## Formatierung der Ausgabe:
<hr>
<h1>Kontierungsanalyse</h1>
<h2>Geschäftsfall: [BEZEICHNUNG]</h2>
<h3>Geschäftsfall-Kategorie: [GESSCÄFTSFALL-KATEGORIE]</h3>
<p>Konto-Soll: [KONTO-NR] - [BEZEICHNUNG] AN Konto-Haben: [KONTO-NR] - [BEZEICHNUNG] [BETRAG] [WÄHRUNG]</p>
<!-- Weitere Kontierungszeilen bei Bedarf -->
<hr>

Beispiel:
<hr>
<h2>Geschäftsfall: Auflösung von Rückstellungen für Instandhaltung</h2>
<h3>Geschäftsfall-Kategorie: Auflösung von Rückstellungen</h3>
<p>Konto-Soll: L3909101 - LC Sonstige Rückstellungen AN Konto-Haben: L160501 - LC-Instandhaltungskosten (Gebäude) 20.000 EUR</p>
<p>Konto-Soll: L160501 - LC Instandhaltungskosten (Gebäude) AN Konto-Haben: L5481101 - LC-Ertrag aus Rückstellungsauflösung 500 EUR</p>
<hr>

Wichtig: Liefere das Ergebnis direkt als HTML ohne Code-Block-Markierungen.

### Schritt 3 - Prüfe die Ergebnisse

## Vorgehen:
Prüfe die Ausgabe entsprechend der oben ausgeführten Schritte und achte darauf, dass: 
- die Informationen klar und strukturiert präsentiert werden,
- die Kontierungsinformationen vollständig und korrekt sind,
- die Ausgabe keine überflüssigen Informationen enthält,
- die Ausgabe keine persönlichen Daten oder sensiblen Informationen enthält,
- die Ausgabe keine nicht relevanten Informationen enthält.



"""

print("REDUCE_Prompt set")

REDUCE_Prompt set


In [ ]:
# function B3.1 - answer: create relevant accounting assigments for business case with LLM (MAP-REDUCE  )
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# Input-variables: map_prompt_template, combine_prompt_template, ui_input
# Output-variables: answer

# set input-variable from ui
question = ui_input

# retrieval from SAP HANA DB with prompt using chain_type="map_reduce" (map: search all relevant docs, reduce: summarize relevant docs to answer )

# create question (map)- and combine (reduce) prompt as chain_type_kwargs
chain_type_kwargs = {
    "question_prompt": PromptTemplate(
        template=map_prompt_template,
        input_variables=["context", "question"]
    ),
    "combine_prompt": PromptTemplate(
        template=combine_prompt_template,
        input_variables=["summaries", "question"]
    )
}

# init retriever with SAP HANA Database
retriever = hana_database.as_retriever(search_kwargs={'k': 20})

# init retrieval-instance 

question_answer_retriever = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_reduce",
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
    verbose=True
)

answer = question_answer_retriever.run(question)
print(answer)



> Entering new RetrievalQA chain...


### function B2: retrieve relevant accounting assignments from SAP HANA vector database
### function B3: create relevant accounting assigments for business case with LLM
#### Version 2 - RetrievalQA-Chain with Type STUFF

In [66]:
# function B2 - retrieve: retrieve relevant accounting assignments from SAP HANA vector database 
# function B2.2.1 - define Prompt (version 1 - light)

# Input-variables: none
# Output-variables: prompt_template

from langchain.prompts import PromptTemplate

prompt_template = """

    Verwende den folgenden Kontext, um die Frage am Ende zu beantworten. Wenn du die Antwort nicht kennst,
    sage einfach, dass du es nicht weißt. Versuche nicht, eine Antwort zu erfinden. 
    Formatiere die Ergebnisse als Liste von JSON-Elementen mit den folgenden Schlüsseln:

    "Geschäftsfall",
    "Konto Soll",
    "Konto Haben"

    Füge keine JSON-Markdown-Codeblock-Syntax in die Ergebnisse ein.

    {context}

    Frage: {question}
"""

prompt_template = PromptTemplate(template=prompt_template, 
                       input_variables=["context", "question"]
                      )

print("Prompt set")

Prompt set


In [ ]:
# function B2 - retrieve: retrieve relevant accounting assignments from SAP HANA vector database 
# function B2.2.2 - define Prompt (version 2 - extended)

# Input-variables: none
# Output-variables: prompt_template

from langchain.prompts import PromptTemplate

prompt_template = """

# Kontierungsanalyse für Geschäftsfälle

Extrahiere kontierungsrelevante Informationen aus folgendem Geschäftsfall:

{context}

## Analyseprozess:

### 1. Analyse der Geschäftsfall-Kategorie
- Identifiziere die Kategorie des Geschäftsfalls 
Geschäftsfall-Kategorie sind sind zum Beispiel:

|Geschäftsfall-Kategorie|
|-----------------------|
|Warenverkauf auf Rechnung|
|Wareneinkauf auf Rechnung|
|Zahlung an Lieferanten|
|Zahlungseingang von Kunden|
|Lohn- und Gehaltszahlungen|
|Anschaffung von Anlagevermögen|
|Abschreibungen|
|Bildung von Rückstellungen|
|Auflösung von Rückstellungen|
|Zahlung der Umsatzsteuer|

- Bestimme die wesentlichen finanziellen Merkmale und Beträge

### 2. Kontierungsregel anwenden
Beachte die Grundprinzipien:
- Jede Buchung erfordert mindestens ein Soll- und Haben-Konto
- Summe Soll = Summe Haben

Ordne den Geschäftsfall einer dieser Kategorien zu:

|Geschäftsfall-Kategorie|Konto Soll|Konto Haben|
|-----------------------|----------|-----------|
|Warenverkauf auf Rechnung|Forderungen aus Lieferungen und Leistungen|Umsatzerlöse und Umsatzsteuer|
|Wareneinkauf auf Rechnung|Wareneinsatz/Materialaufwand und Vorsteuer|Verbindlichkeiten aus Lieferungen und Leistungen|
|Zahlung an Lieferanten|Verbindlichkeiten aus Lieferungen und Leistungen|Bank oder Kasse|
|Zahlungseingang von Kunden|Bank oder Kasse|Forderungen aus Lieferungen und Leistungen|
|Lohn- und Gehaltszahlungen|Personalaufwand|Bank und diverse Verbindlichkeiten (Lohnsteuer, Sozialversicherung)|
|Anschaffung von Anlagevermögen|Anlagevermögen und Vorsteuer|Bank oder Verbindlichkeiten|
|Abschreibungen|Abschreibungsaufwand|Kumulierte Abschreibungen (Wertberichtigung Anlagevermögen)|
|Bildung von Rückstellungen|Aufwand für Rückstellungen|Rückstellungen|
|Auflösung von Rückstellungen|Rückstellungen|Sonstige betriebliche Erträge oder Aufwandskonto|
|Zahlung der Umsatzsteuer|Umsatzsteuer|Bank|


### 3. Relevante Kontierung extrahieren
- Ermittle konkrete Konto-Nummern und -Bezeichnungen aus dem Kontext
- Bestimme die exakten Buchungsbeträge (inkl. Steuern falls zutreffend)
- Identifiziere eventuelle Besonderheiten oder Ausnahmen

### 4. Qualitätsprüfung
- Prüfe die doppelte Buchführung (Betrag Soll = Betrag Haben)
- Stelle Übereinstimmung mit gesetzlichen Anforderungen sicher
- Verifiziere die inhaltliche Korrektheit der Kontierung

## Ausgabe:
- Geschäftsfall-Kategorie
- Exakte Kontierungsinformation mit Kontonummern und -bezeichnungen
- Soll-Haben-Beziehung mit Beträgen

Falls keine eindeutigen Kontierungsinformationen extrahierbar: "Keine ausreichenden Kontierungsinformationen im Kontext verfügbar."

Frage: {question}

"""

prompt_template = PromptTemplate(template=prompt_template, 
                       input_variables=["context", "question"]
                      )

print("Prompt set")

In [68]:
# function B2 - retrieve: retrieve relevant accounting assignments from SAP HANA vector database 
# function B2.2.3 - define Prompt (version 2 - extended with output-Format)

# Input-variables: none
# Output-variables: prompt_template

from langchain.prompts import PromptTemplate

prompt_template = """

# Kontierungsanalyse für Geschäftsfälle

Extrahiere kontierungsrelevante Informationen aus folgendem Geschäftsfall:

{context}

Gehe dabei Schritt für Schritt vor und achte auf die folgenden Punkte:
- Identifiziere die Kategorie des Geschäftsfalls    
- Bestimme die wesentlichen finanziellen Merkmale und Beträge
- Wende die Kontierungsregel an
- Ermittle konkrete Konto-Nummern und -Bezeichnungen aus dem Kontext
- Bestimme die exakten Buchungsbeträge (inkl. Steuern falls zutreffend)
- Identifiziere eventuelle Besonderheiten oder Ausnahmen
- Nutze folgenden Schritte des Analyseprozesses

## Analyseprozess:

### 1. Identifiziere die Geschäftsfall-Kategorie
- Identifiziere die Kategorie des Geschäftsfalls 
Geschäftsfall-Kategorie sind sind zum Beispiel:

|Geschäftsfall-Kategorie|
|-----------------------|
|Warenverkauf auf Rechnung|
|Wareneinkauf auf Rechnung|
|Zahlung an Lieferanten|
|Zahlungseingang von Kunden|
|Lohn- und Gehaltszahlungen|
|Anschaffung von Anlagevermögen|
|Abschreibungen|
|Bildung von Rückstellungen|
|Auflösung von Rückstellungen|
|Zahlung der Umsatzsteuer|

- Bestimme die wesentlichen finanziellen Merkmale und Beträge

### 2. Wende die Kontierungsregel an

Beachte die Grundprinzipien:
- Jede Buchung erfordert mindestens ein Soll- und Haben-Konto
- Summe Soll = Summe Haben

Ordne den Geschäftsfall anhand des Kontext einer dieser Geschäftsfall-Kategorien zu:

|Geschäftsfall-Kategorie|Konto Soll|Konto Haben|
|-----------------------|----------|-----------|
|Warenverkauf auf Rechnung|Forderungen aus Lieferungen und Leistungen|Umsatzerlöse und Umsatzsteuer|
|Wareneinkauf auf Rechnung|Wareneinsatz/Materialaufwand und Vorsteuer|Verbindlichkeiten aus Lieferungen und Leistungen|
|Zahlung an Lieferanten|Verbindlichkeiten aus Lieferungen und Leistungen|Bank oder Kasse|
|Zahlungseingang von Kunden|Bank oder Kasse|Forderungen aus Lieferungen und Leistungen|
|Lohn- und Gehaltszahlungen|Personalaufwand|Bank und diverse Verbindlichkeiten (Lohnsteuer, Sozialversicherung)|
|Anschaffung von Anlagevermögen|Anlagevermögen und Vorsteuer|Bank oder Verbindlichkeiten|
|Abschreibungen|Abschreibungsaufwand|Kumulierte Abschreibungen (Wertberichtigung Anlagevermögen)|
|Bildung von Rückstellungen|Aufwand für Rückstellungen|Rückstellungen|
|Auflösung von Rückstellungen|Rückstellungen|Sonstige betriebliche Erträge oder Aufwandskonto|
|Zahlung der Umsatzsteuer|Umsatzsteuer|Bank|


### 3. Extrahiere die relevante Kontierung aus dem Kontext
- Ermittle konkrete Konto-Nummern und -Bezeichnungen aus dem Kontext
- Bestimme die exakten Buchungsbeträge (inkl. Steuern falls zutreffend)
- Identifiziere eventuelle Besonderheiten oder Ausnahmen

### 4. Prüfe die Qualität des Ergebnisses
- Prüfe die doppelte Buchführung (Betrag Soll = Betrag Haben)
- Stelle Übereinstimmung mit gesetzlichen Anforderungen sicher
- Verifiziere die inhaltliche Korrektheit der Kontierung

### 5. Gib die Ergebnisse aus

## Vorgehen:
1. Entferne Duplikate und redundante Informationen
2. Priorisiere die relevantesten und spezifischsten Kontierungen
3. Strukturiere das Ergebnis klar und übersichtlich

- Gib folgende Informationen des Ergebnisses aus:
    * Geschäftsfallbezeichnung
    * Geschäftsfall-Kategorie
    * Exakte Kontierungsinformation mit Kontonummern und -bezeichnungen
    * Soll-Haben-Beziehung mit Beträgen

- Formatiere die Ausgabe in HTML wie folgt:
    <hr>
    <h2>Geschäftsfall: [BEZEICHNUNG]</h2>
    <h3>Geschäftsfall-Kategorie: [KATEGORIE]</h3>
    <h3>Kontierung</h3>
    <p>Konto-Soll: [KONTO-NR] - [BEZEICHNUNG] AN Konto-Haben: [KONTO-NR] - [BEZEICHNUNG] [BETRAG] [WÄHRUNG]</p>
    <!-- Weitere Kontierungszeilen bei Bedarf -->
    <hr>

    Beispiel:
    <hr>
    <h2>Geschäftsfall: Auflösung von Rückstellungen</h2>
    <h3>Kontierung</h3>
    <p>Konto-Soll: L3909101 - LC Sonstige Rückstellungen AN Konto-Haben: L160501 - LC-Instandhaltungskosten (Gebäude) 20.000 EUR</p>
    <p>Konto-Soll: L160501 - LC Instandhaltungskosten (Gebäude) AN Konto-Haben: L5481101 - LC-Ertrag aus Rückstellungsauflösung 500 EUR</p>
    <hr>

    Falls keine eindeutigen Kontierungsinformationen gefunden wurden: "Ich habe keine Kontierungsinformationen zum Geschäftsfall gefunden."

    Wichtig: Liefere das Ergebnis direkt als HTML ohne Code-Block-Markierungen.

Prüfe die Ausgabe entsprechend der oben ausgeführten Schritte und achte darauf, dass: 
- die Informationen klar und strukturiert präsentiert werden,
- die Ausgabe in HTML-Format erfolgt und keine Code-Block-Markierungen enthält,
- die Kontierungsinformationen vollständig und korrekt sind,
- die Ausgabe keine überflüssigen Informationen enthält,
- die Ausgabe keine persönlichen Daten oder sensiblen Informationen enthält,
- die Ausgabe keine nicht relevanten Informationen enthält.

Frage: {question}

"""

prompt_template = PromptTemplate(template=prompt_template, 
                       input_variables=["context", "question"]
                      )

print("Prompt set")

Prompt set


In [71]:
# function B3.2 - answer: create relevant accounting assigments for business case with LLM

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# Input-variables: prompt
# Output-variables: 

# set parameters
count_retrieved_documents = 40 # hint:  smaller than 20 -> to much tokens

# set input-variable from ui
question = input_manager.get_current_input()
chain_type_kwargs = {"prompt": prompt_template}

# init retriever with SAP HANA Database
retriever = hana_database.as_retriever(search_kwargs={'k': count_retrieved_documents})


question_answer_retriever = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs=chain_type_kwargs,
    verbose=True
)

answer = question_answer_retriever.run(question)
print(answer)



> Entering new RetrievalQA chain...

> Finished chain.
<hr>
<h2>Geschäftsfall: Bildung und Auflösung von Rückstellungen für Instandhaltungsmaßnahmen</h2>
<h3>Geschäftsfall-Kategorie: Bildung von Rückstellungen</h3>
<h3>Kontierung</h3>
<p>Konto-Soll: L160501 - LC Instandhaltungskosten (Gebäude) AN Konto-Haben: L3909101 - LC Sonstige Rückstellungen 20.000 EUR</p>
<hr>
<h3>Geschäftsfall-Kategorie: Auflösung von Rückstellungen</h3>
<h3>Kontierung</h3>
<p>Konto-Soll: L3909101 - LC Sonstige Rückstellungen AN Konto-Haben: L61605010 - Instandhaltungskosten 19.500 EUR</p>
<p>Konto-Soll: 15760000 - Vorsteuer AN Konto-Haben: 44001000 - Kreditor 3.705 EUR</p>
<p>Konto-Soll: 44001000 - Kreditor AN Konto-Haben: 12000000 - Bank 23.205 EUR</p>
<hr>


In [ ]:
# function B4 - output: give answer with accounting assigments to user
# use of user interface function B1

# set answer to output field
input_manager.update_output(answer)

# display user interface (Widget)
input_manager.display_widget()

### Scrap-Book Development Modul B

* UI-Object version 1 (code)
* UI-Object version 2 (class)
* function A4.3: Check retrieval for the embeddings in the SAP-Hana-Database
* check function B2/3: check retrieval from SAP HANA DB with prompt using chain_type="map_reduce"
* check function B2/3: check retrieval from SAP HANA DB with prompt using chain_type="stuff"
* check function B2/3: check retrieval from SAP HANA DB with prompt using chain_type="map_reduce" and optimized prompt
* Example setup RAG (wichtige Punkte - cursor)

In [ ]:
# function B1.1 - setup class user interface
# user interface version 2 (code) function B1.1 : get booking information business case from user
# the function show ui and store input from user 

import ipywidgets as widgets
from IPython.display import display, clear_output

# Chat Interface erstellen
input_text = widgets.Text(
    value='',
    placeholder='Geben Sie Ihre Buchungsinformationen ein...',
    description='Eingabe:',
    disabled=False,
    layout=widgets.Layout(width='80%')
)

output_text = widgets.Output(
    layout=widgets.Layout(width='80%', border='1px solid black', padding='10px')
)

def on_submit(b):
    with output_text:
        clear_output()
        print(f"Ihre Eingabe: {input_text.value}")
        # Hier können später die Verarbeitungsfunktionen aufgerufen werden
        input_text.value = ''  # Eingabefeld leeren

submit_button = widgets.Button(
    description='Senden',
    button_style='primary',
    tooltip='Klicken Sie hier, um die Eingabe zu senden'
)

submit_button.on_click(on_submit)

# Layout erstellen
vbox = widgets.VBox([
    widgets.HTML('<h3>Digitaler Buchungsassistent - Kontierungshilfe</h3>'),
    input_text,
    submit_button,
    output_text
])

display(vbox)


In [ ]:
# function B1.2 - input version 1 (full object) : get booking information business case from user
# the function uses a UI-object for storing input from user 


import ipywidgets as widgets
from IPython.display import display, HTML

class InputManager:
    def __init__(self):
        self.input_text = ""
        self.history = []
        
        # Button erstellen
        self.button = widgets.Button(
            description='Senden',
            button_style='primary',
            layout=widgets.Layout(
                width='150px',
                height='40px',
                margin='0 0 10px 0'
            )
        )
        
        # Eingabebereich mit Überschrift
        input_label = widgets.HTML(
            value="<h3 style='margin-bottom: 10px; color: #333;'>Eingabe Buchungsinformationen</h3>"
        )
        
        # Widget für die Texteingabe erstellen
        self.text_input = widgets.Textarea(
            value='',
            placeholder='Geben Sie hier Ihre Buchungsinformationen ein...',
            disabled=False,
            layout=widgets.Layout(
                width='800px',
                height='200px',
                # overflow='hidden'  # Verhindert Scrolling
            )
        )
        
        # Ausgabebereich mit Überschrift
        output_label = widgets.HTML(
            value="<h3 style='margin: 20px 0 10px 0; color: #333;'>Ausgabe Kontierungsinformationen</h3>"
        )
        
        # Ausgabefeld erstellen mit fester Höhe und ohne Scroll
        self.output = widgets.HTML(
            value="""
            <div style='
                border: 1px solid #ddd;
                padding: 10px;
                background-color: white;
                height: 300px;
                width: 800px;
            '>
                <p>Kontierungsinformationen werden hier ausgegeben ...</p>
            </div>
            """,
            layout=widgets.Layout(
                width='800px',
                height='150px',
                # overflow='hidden'  # Verhindert Scrolling
            )
        )
        
        # Button-Callback definieren
        self.button.on_click(self.on_button_click)
        
        # Vertikales Layout erstellen
        content = widgets.VBox([
            input_label,
            self.button,
            self.text_input,
            output_label,
            self.output
        ], layout=widgets.Layout(
            width='900px',
            align_items='flex-start'
        ))
        
        # Gesamtcontainer erstellen
        self.container = widgets.VBox([
            widgets.HTML("<h1>Digitaler Buchungsassistent - Kontierungshilfe</h1>"),
            content
        ], layout=widgets.Layout(
            width='900px',
            padding='20px'
        ))
    
    def on_button_click(self, b):
        """Wird aufgerufen, wenn der Button geklickt wird"""
        self.input_text = self.text_input.value
        self.history.append(self.input_text)
        
        # Ausgabefeld aktualisieren
        self.update_output(f"Eingabe: {self.input_text}")
        
        # Eingabefeld leeren nach Absenden
        # self.text_input.value = ''
    
    def update_output(self, text):
        """Ausgabefeld aktualisieren"""
        self.output.value = f"""
        <div style='
            border: 1px solid #ddd;
            padding: 10px;
            background-color: white;
            height: 300px;
            width: 800px;
            # overflow: hidden;
        '>
            <p>{text}</p>
        </div>
        """
    
    def display_widget(self):
        """Widget anzeigen"""
        display(self.container)
    
    def get_current_input(self):
        """Aktuelle Eingabe zurückgeben"""
        return self.input_text
    
    def get_history(self):
        """Eingabehistorie zurückgeben"""
        return self.history
    
    def save_history(self, filename='input_history.json'):
        """Historie in Datei speichern"""
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(self.history, f, ensure_ascii=False, indent=2)
    
    def load_history(self, filename='input_history.json'):
        """Historie aus Datei laden"""
        try:
            with open(filename, 'r', encoding='utf-8') as f:
                self.history = json.load(f)
        except FileNotFoundError:
            print("Keine gespeicherte Historie gefunden.")

# Instanz erstellen
input_manager = InputManager()

# Widget anzeigen
input_manager.display_widget()

In [ ]:
# function B1.2 - input version 2 (full object) : store booking information business case from user
# examples function call for calling information from ui-class

# call current input
ui_current_input = input_manager.get_current_input()

# call History
history = input_manager.get_history()

# store History in file
input_manager.save_history()

# load History from file
input_manager.load_history()

# change content Output-field every time
input_manager.update_output("Aktualisieurng der Nachricht...")

# In jeder anderen Zelle verfügbar
print("Aktuelle Eingabe:", input_manager.input_text)
print("Variable mit Input-Inhalt:", ui_current_input)
print("Alle bisherigen Eingaben:", input_manager.history)

In [ ]:
# check function B2/3: Check retrieval for the embeddings in the SAP-Hana-Database

import os
from langchain import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = prompt | llm

question = "What NFL team won the Super Bowl in the year Justin Bieber was born?"

print(llm_chain.invoke({'question': question}))
answer = llm_chain.invoke({'question': question})
print(answer)


In [ ]:
# check function B2/3: check retrieval from SAP HANA DB with prompt using chain_type="map_reduce"

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

map_template = """
Analysiere den folgenden Kontext und extrahiere relevante Informationen zur Kontierung:

{context}

Frage: {question}

Gib die relevanten Informationen in einem kurzen Zwischenergebnis zurück.

"""

reduce_template = """
Basierend auf den folgenden Zwischenergebnissen, erstelle eine finale Antwort:

{summerization}

Frage: {question}

Formatiere die Ergebnisse in einer Liste von JSON-Elementen mit den folgenden Schlüsseln:
"Geschäftsfall"
"Konto Soll"
"Konto Haben"

Die Ergebnisse dürfen keine json markdown codeblock syntax enthalten.
Wenn keine relevanten Informationen gefunden wurden, gib an dass Du keine Antwort kennst.
"""

MAP_PROMPT = PromptTemplate(template=map_template, input_variables=["context", "question"])
REDUCE_PROMPT = PromptTemplate(template=reduce_template, input_variables=["summerization", "question"])

chain_type_kwargs = {
    "question_prompt": MAP_PROMPT,
    "reduce_prompt": REDUCE_PROMPT
}

#question = "Finde Kontierung für die Umbuchung von langfristigen Forderungen"

question = input_manager.get_current_input()

retriever = hana_database.as_retriever(search_kwargs={'k':20})

question_answer_retriever = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type='map_reduce',
     verbose=True
)

answer = question_answer_retriever.run(question)
print(answer)

In [ ]:
# check function B2/3: : check retrieval from SAP HANA DB with prompt using chain_type="stuff"
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

prompt_template = """Verwende den folgenden Kontext, um die Frage am Ende zu beantworten. Wenn du die Antwort nicht kennst,
    sage einfach, dass du es nicht weißt. Versuche nicht, eine Antwort zu erfinden. 
    Formatiere die Ergebnisse als Liste von JSON-Elementen mit den folgenden Schlüsseln:

    "Geschäftsfall",
    "Konto Soll",
    "Konto Haben"

    Füge keine JSON-Markdown-Codeblock-Syntax in die Ergebnisse ein.

    {context}

    Frage: {question}
"""

PROMPT = PromptTemplate(template=prompt_template, 
                       input_variables=["context", "question"]
                      )
    
chain_type_kwargs = {"prompt": PROMPT}

question = "Finde Kontierung für die Buchung von Rückstellungen"

count_retrieved_documents = 10

retriever = hana_database.as_retriever(search_kwargs={'k': count_retrieved_documents})
# hint: k smaller than 20 -> to much tokens

question_answer_retriever = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs=chain_type_kwargs,
    verbose=True
)

answer = question_answer_retriever.run(question)
print(answer)

In [ ]:
# check function B2/3 : check retrieval from SAP HANA DB with prompt using chain_type="map_reduce" (prompt-finetuning)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# Prompt für die Map-Phase
map_prompt_template = """Analysiere den folgenden Kontext und extrahiere relevante Kontierungsinformationen für den Geschäftsfall.
Wenn keine relevanten Informationen im Kontext gefunden werden können, gib die Antwort zurück: "Ich habe keine Kontierungsinformationen zum Geschäftsfall gefunden".

{context}

Frage: {question}
"""

# Prompt für die Combine/Reduce-Phase
combine_prompt_template = """
- Fasse die folgenden Kontierungsinformationen zusammen und entferne Duplikate.
- Gib nur die relevantesten und eindeutigsten Kontierungen zurück. 
- Wenn keine relevanten Informationen im Kontext gefunden werden können, gib die Antwort zurück: "Ich habe keine Kontierungsinformationen zum Geschäftsfall gefunden".
- Wenn relevante Informationen gefunden wurden gib diese Informationen im folgenden Struktur aus:
    ## Geschäftsfall: <Bezeichnung des Geschäftsfalls>
    ## Kontierung
    Konto-Soll: <Konto-Soll> - <Bezeichnung Konto-Soll> AN Konto-Haben: <Konto-Haben> - <Bezeichnung Konto-Haben>

Beispiel:

-----------------------------
## Geschäftsfall: Bildung von Rückstellungen
    ## Kontierung
    Konto-Soll: L160501 - LC Instandhaltungskosten (Gebäude) AN Konto-Haben: L3909101 - LC Sonstige Rückstellungen
-----------------------------

{summaries}

Frage: {question}
"""

# Korrekte Struktur für chain_type_kwargs
chain_type_kwargs = {
    "question_prompt": PromptTemplate(
        template=map_prompt_template,
        input_variables=["context", "question"]
    ),
    "combine_prompt": PromptTemplate(
        template=combine_prompt_template,
        input_variables=["summaries", "question"]
    )
}

question = "Finde Kontierung für die Buchung von Bildung von Rückstellungen"

retriever = hana_database.as_retriever(search_kwargs={'k': 20})

question_answer_retriever = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_reduce",
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
    verbose=True
)

answer = question_answer_retriever.run(question)
print(answer)

### Example setup RAG (wichtige Punkte - cursor)

Wichtige Punkte:
1. Umgebungsvariablen:
* Stellen Sie sicher, dass alle HANA-Verbindungsparameter korrekt gesetzt sind
* Der OpenAI API Key muss verfügbar sein
2. HANA Setup:
* Passen Sie den table_name an Ihre tatsächliche Tabelle an
* Die embedding_dimension ist für das ada-002 Modell auf 1536 gesetzt
3. Prompts:
* Beide Prompts sind für das Map-Reduce-Verfahren konfiguriert
* Die Ausgabe wird im JSON-Format strukturiert
4.Fehlerbehandlung:
* Der Code enthält grundlegende Fehlerbehandlung
* Detaillierte Fehlerausgabe für Debugging

Um den Code auszuführen, müssen Sie sicherstellen, dass:
Alle erforderlichen Pakete installiert sind
Die Umgebungsvariablen korrekt gesetzt sind
Die HANA-Datenbank erreichbar ist
Die Vektortabelle existiert und korrekt konfiguriert ist

In [ ]:
# example setup RAG (cursor)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import os
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores.hanavector import HanaDB

# OpenAI API Key sollte bereits gesetzt sein
# Falls nicht, hier setzen:
# os.environ["OPENAI_API_KEY"] = "your-api-key"

# LLM Model initialisieren
llm = ChatOpenAI(
    model_name="gpt-4o",
    temperature=0
)

# Embedding Model initialisieren
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

# HANA Verbindungsparameter
connection_params = {
    'address': os.getenv('HDB_HOST'),
    'port': os.getenv('HDB_PORT'),
    'user': os.getenv('HDB_USER'),
    'password': os.getenv('HDB_PASSWORD'),
    'schema': os.getenv('HDB_SCHEMA')
}

# HANA Datenbank initialisieren
hana_database = HanaDB(
    embedding=embeddings,
    connection_params=connection_params,
    table_name="VECTOR_TABLE",  # Ihr Tabellenname
    embedding_dimension=1536    # Dimension für text-embedding-ada-002
)

# Map Prompt für die erste Phase
map_prompt_template = """Analysiere den folgenden Kontext und extrahiere relevante Kontierungsinformationen für den Geschäftsfall.
Formatiere die Ergebnisse als Liste von JSON-Elementen mit den Schlüsseln "Geschäftsfall", "Konto Soll", "Konto Haben".

Kontext: {context}

Frage: {question}
"""

# Reduce Prompt für die Zusammenfassung
reduce_prompt_template = """Fasse die folgenden Kontierungsinformationen zusammen und entferne Duplikate.
Gib nur die relevantesten und eindeutigsten Kontierungen zurück.
Formatiere das Ergebnis als Liste von JSON-Elementen mit den Schlüsseln "Geschäftsfall", "Konto Soll", "Konto Haben".

Kontierungsinformationen:
{context}

Frage: {question}
"""

# Prompts erstellen
MAP_PROMPT = PromptTemplate(
    template=map_prompt_template,
    input_variables=["context", "question"]
)

REDUCE_PROMPT = PromptTemplate(
    template=reduce_prompt_template,
    input_variables=["context", "question"]
)

# Chain Type kwargs konfigurieren
chain_type_kwargs = {
    "question_prompt": MAP_PROMPT,
    "combine_prompt": REDUCE_PROMPT
}

# Retriever konfigurieren
retriever = hana_database.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 20}
)

# RetrievalQA Chain erstellen
question_answer_retriever = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='map_reduce',
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
    verbose=True
)

# Beispielfrage
question = "Finde Kontierung für die Buchung von Rückstellungen"

try:
    # Antwort generieren
    answer = question_answer_retriever.run(question)
    print("Antwort:", answer)
except Exception as e:
    print("Ein Fehler ist aufgetreten:", str(e))
    # Detailliertere Fehlerinformationen
    import traceback
    print("\nDetaillierter Fehler:")
    print(traceback.format_exc())